# Detection using new YOLOv3 model

In [2]:
# Import the required Libraries
from tkinter import *
from tkinter import ttk, filedialog
import cv2
import numpy as np
from tkinter.filedialog import askopenfile

net = cv2.dnn.readNet('yolov3_custom_7000.weights', 'yolov3_testing.cfg')
classes = []
with open("classes.txt", "r") as f:
    classes = f.read().splitlines()
    
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

# Create an instance of tkinter frame
win = Tk()

# Set the geometry of tkinter frame
win.geometry("700x350")

count = 0
color ="g"

def open_file():
   file = filedialog.askopenfile(mode='r', filetypes=[('videos', '*.mp4')])
   if file:
      cap = cv2.VideoCapture(file.name)

      # Check if camera opened successfully
      if (cap.isOpened() == False):
         print("Error opening video  file")

      # Read until video is completed
      while (cap.isOpened()):

         # Capture frame-by-frame
         ret, frame = cap.read()
         if ret == True:
            img = cv2.resize(frame,(1000,720),fx=0,fy=0,interpolation = cv2.INTER_CUBIC)
            height, width, _ = img.shape
            blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
            net.setInput(blob)
            output_layers_names = net.getUnconnectedOutLayersNames()
            layerOutputs = net.forward(output_layers_names)

            boxes = []
            confidences = []
            class_ids = []

            for output in layerOutputs:
                for detection in output:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.2:
                        center_x = int(detection[0]*width)
                        center_y = int(detection[1]*height)
                        w = int(detection[2]*width)
                        h = int(detection[3]*height)

                        x = int(center_x - w/2)
                        y = int(center_y - h/2)

                        boxes.append([x, y, w, h])
                        confidences.append((float(confidence)))
                        class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

            if len(indexes)>0:
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    confidence = str(round(confidences[i],2))
                    color = colors[i]
                    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                    cv2.putText(img, label + " " + confidence, (x, y+20), font, 1, (255,255,255), 2)

 

            # Display the resulting frame
            cv2.imshow('Frame', img)

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
               break

         # Break the loop
         else:
            break

      # When everything done, release
      # the video capture object
      cap.release()

# Add a Label widget
label = Label(win, text="Click the Button to browse the Files", font=('Georgia 13'))
label.pack(pady=10)


# Create a Button
ttk.Button(win, text="Browse", command=open_file).pack(pady=20)

win.mainloop()
